In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

In [2]:
def auto_predict(data_x,data_set,cat,example_show):
    pred = model.predict(data_x)
    y_test = cat_decode(pred,cat)
    data_set['Expected']= y_test
    data_set.to_csv('sample_submission_1234.csv',columns=['Id','Expected'],index=False)
    if example_show==True:
        return print("Exito!\n",y_test[0:20])
    else:
        return print("Exito")

In [3]:
def txt_extract(names_data,path):
    data_list=[]
    for i in names_data:
        sentence = ""
        data_path = path + str(i)+".txt"
        text= open(data_path,'r')
        for x in text:
            sentence+=str(x)
        data_list.append(sentence)
        
        text.close()
    return data_list


In [9]:
def txt_original_extract(name):
    original_list=[]
    data=open(name,'r')
    data_read=[line.rstrip('\n') for line in data]
    for x in data_read:
        aux_list=[]
        x_aux=x.split("\t",1)
        original_id=x_aux[0]
        x_aux.remove(original_id)
        x_aux = " ".join(str(w) for w in x_aux)
        aux_list.append(original_id)
        aux_list.append(x_aux)
        original_list.append(aux_list)
    data.close()
    return original_list

In [ ]:
def cat_encode(data,cat):
    data=list(data)
    cat=list(cat)
    for i in range(len(data)):
        for j in range(len(cat)):
            if data[i]==cat[j]:
                data[i]=j
                break
            else:
                continue
    return keras.utils.to_categorical(data,len(cat))
    #return np.asarray(data)
def cat_decode(data,cat):
    #Y=data
    Y= [np.argmax(y, axis=None, out=None) for y in data]
    cat=list(cat)
    
    for i in range(len(Y)):
        Y[i]=cat[Y[i]]
    return Y

## Loading data

In [29]:
train_labels = pd.read_csv('./data/train_labels.csv')
test_labels = pd.read_csv('./data/sample_submission.csv')
train_message_path= "./data/train_source_tweets.txt"
train_path = './data/train/'
test_path = './data/test/'

###########        This creates original df with original message and label      ##################33
x_original = txt_original_extract(train_message_path)
df_original = pd.DataFrame(x_original,  columns =['id','original message'])
df_original['id']= df_original['id'].astype('int64')
df_train = pd.merge(df_original, train_labels, how='inner', left_on='id', right_on='id')
df_train.head(50)
##############################################################################33

,id,original message,label
0,692735698349199360,north korea 'planning some kind of rocket laun...,non-rumor
1,525008463819464704,"meet kevin vickers, the hero who shot down the...",unverified
2,505611045897924608,"15 year old who ""swatted"" gamer convicted of d...",false
3,693466724822323200,audio recordings reveal cpr started 11 minutes...,non-rumor
4,510922415468449792,awful mt @scclemons:uk aid worker david haines...,true
5,549975297160802304,dammit - a transgender 17-year-old left a suic...,unverified
6,500378522788315137,#ferguson police: officer involved in fatal sh...,unverified
7,528607634099015680,#animaloftheday: the first afghan fanged deer ...,true
8,489854703321509888,"'nine britons, 23 us citizens and 80 children'...",false
9,521444489697583104,photos proved that terrorists used chemical we...,unverified


### Extracting train data

In [32]:
train_message = df_train["original message"].values
train_relations = txt_extract(df_train["id"],train_path )
y_train = df_train["label"]
cat = Y_train.unique()
Y_train=cat_encode(Y_train,cat)

# Model

# Compile

# Results